In [0]:
import pyspark.sql.functions as F
brand_table = spark.table("ecommerce.silver.brand_base_table")
category_table = spark.table("ecommerce.silver.category_base_table")
product_table = spark.table("ecommerce.silver.product_base_table")
brand_table.createOrReplaceTempView("brand_table")
product_table.createOrReplaceTempView("product_table")
category_table.createOrReplaceTempView("category_table")

# Creating the Product Gold table

In [0]:
%sql
use catalog ecommerce;
CREATE OR REPLACE TABLE gold.product_dim AS

select 
p.product_id, p.sku, b.brand_code , b.brand_name , c.category_code , c.category_name , p.color,p.size,p.material,p.weight_grams,p.length_cm, p.width_cm,p.height_cm,p.rating_count from brand_table b left join category_table c on b.category_code = c.category_name left join product_table p on p.brand_code = b.brand_code


# Creating Customer Gold Table



In [0]:
from pyspark.sql import Row

india_region = {
    "MH": "West", "GJ": "West", "RJ": "West",
    "KA": "South", "TN": "South", "TS": "South", "AP": "South", "KL": "South",
    "UP": "North", "WB": "North", "DL": "North"
}
australia_region = {
    "VIC": "SouthEast",  "NSW": "East", "QLD": "NorthEast" , "WA": "West"
}
uk_region = {
    "ENG": "England", "WLS": "Wales", "NIR": "Northern Ireland", "SCT": "Scotland"
}
us_region = {
    "MA": "NorthEast", "FL": "South", "NJ": "NorthEast", "CA": "West", 
    "NY": "NorthEast", "TX": "South" ,"WA": "West"
}
uae_region = {
    "AUH": "Abu Dhabi", "DU": "Dubai", "SHJ": "Sharjah"
}
singapore_region = {
    "SG": "Singapore"
}
canada_region = {
    "BC": "West", "AB": "West", "ON": "East", "QC": "East", "NS": "East", "IL": "Other"
}

region_map = {
    "India":india_region,
    "United States":us_region,
    "United Kingdom":uk_region,
    "Australia":australia_region,
    "Canada":canada_region,
    "United Arab Emirates":uae_region,
    "Singapore":singapore_region
}

region_rows = []
for country, states in region_map.items():
    for state, region_name in states.items():
        region_rows.append(Row(country=country, region_name=region_name, state=state))

df = spark.createDataFrame(region_rows)
customer_table = spark.table("ecommerce.silver.customer_base_table")
combined_customer = customer_table.join(df,on=['state','country'],how='left')
combined_customer.write.format("delta").mode("overwrite").saveAsTable("ecommerce.gold.customer_dim")

# Creating Date Gold Table


In [0]:
df = spark.table("ecommerce.silver.date_base_table")
df = df.withColumn("is_weekend", F.when(F.col("day_name").isin("SATURDAY","SUNDAY"), 1).
otherwise(0))
df = df.filter(F.col("date").isNotNull())
df = df.withColumn("Month",F.date_format(F.col("date"),"MMMM"))
df = df.withColumn("date_id",F.date_format(F.col("date"),"yyyyMMdd").cast("int"))
df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.gold.date_dim")